In [49]:
import pandas as pd
import numpy as np
import pytz

def calculate_night_hours(file_path):
    df = pd.read_csv(file_path, low_memory=False)
    
    # Ensure that 'startDate' and 'endDate' columns are in datetime format
    df['startDate'] = pd.to_datetime(df['startDate'])
    df['endDate'] = pd.to_datetime(df['endDate'])


    mask_start = df['startDate'].dt.hour >= 6
    mask_end = df['endDate'].dt.hour < 24
    df_filtered = df[mask_start & mask_end]

    df_filtered['date'] = (df_filtered['startDate'] - pd.Timedelta(hours=12)).dt.date
    grouped = df_filtered.groupby('date').agg(startSleep=('startDate', 'min'), endSleep=('endDate', 'max')).reset_index()


    return grouped


features = [
    ('./data/xml_export/HeartRate.csv', "hr"),
    ('./data/xml_export/StepCount.csv', "steps"),
    ('./data/xml_export/DistanceWalkingRunning.csv', "distance"),
    ]



In [50]:
calculate_night_hours('./data/xml_export/AppleStandTime.csv')

/tmp/ipykernel_18730/1404084676.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = (df_filtered['startDate'] - pd.Timedelta(hours=12)).dt.date


,date,startSleep,endSleep
0,2020-09-25,2020-09-25 19:05:00-04:00,2020-09-26 12:00:00-04:00
1,2020-09-26,2020-09-26 12:00:00-04:00,2020-09-27 12:00:00-04:00
2,2020-09-27,2020-09-27 12:00:00-04:00,2020-09-28 11:45:00-04:00
3,2020-09-28,2020-09-28 12:10:00-04:00,2020-09-29 12:00:00-04:00
4,2020-09-29,2020-09-29 12:00:00-04:00,2020-09-30 12:00:00-04:00
...,...,...,...
896,2023-03-13,2023-03-13 18:10:00-04:00,2023-03-14 10:05:00-04:00
897,2023-03-14,2023-03-14 12:55:00-04:00,2023-03-15 10:05:00-04:00
898,2023-03-15,2023-03-15 12:15:00-04:00,2023-03-16 11:15:00-04:00
899,2023-03-16,2023-03-16 15:05:00-04:00,2023-03-17 10:45:00-04:00


In [48]:
stand = pd.read_csv('./data/xml_export/AppleExerciseTime.csv', low_memory=False)

stand['startDate'] = pd.to_datetime(stand['startDate'])
stand['endDate'] = pd.to_datetime(stand['endDate'])

stand["date"] = (stand["startDate"] - pd.Timedelta(hours=12)).dt.date

# hours between shift 1 startDate from endDate
stand["hours_between"] = (stand["startDate"] - stand["endDate"].shift(1)).dt.total_seconds() / 3600
stand = stand.groupby('date').agg(hours_between_total=('hours_between', 'sum')).reset_index()
stand

,date,hours_between_total
0,2021-03-26,0.000000
1,2021-04-09,335.981944
2,2021-10-13,4487.970833
3,2022-01-23,2447.980833
4,2022-02-10,431.983333
5,2022-03-17,839.983333
6,2022-10-14,5063.983333
7,2022-11-18,839.983333


In [66]:
import pandas as pd

df = pd.read_csv('./data/xml_export/AppleStandTime.csv', low_memory=False)

# Assuming your dataframe is called df and has columns 'startDate' and 'endDate'
# Make sure the columns are in datetime format
df['startDate'] = pd.to_datetime(df['startDate']).dt.tz_localize(None)
df['endDate'] = pd.to_datetime(df['endDate']).dt.tz_localize(None)

def find_sleep_intervals(df):
    # Get the date range in the dataframe
    min_date = df['startDate'].min().date()
    max_date = df['endDate'].max().date()

    # Initialize an empty list to store the results
    results = []

    # Loop through each date in the range
    for date in pd.date_range(min_date, max_date):
        # startSleep time boundaries
        start_day = pd.Timestamp.combine(date, pd.Timestamp('20:00:00').time())
        end_day = pd.Timestamp.combine(date + pd.DateOffset(1), pd.Timestamp('03:00:00').time())
        
        # endSleep time boundaries
        start_night = pd.Timestamp.combine(date + pd.DateOffset(1), pd.Timestamp('04:00:00').time())
        end_night = pd.Timestamp.combine(date + pd.DateOffset(1), pd.Timestamp('11:00:00').time())

        # Filter the dataframe for max_endDate
        mask_endDate = (df['endDate'] >= start_day) & (df['endDate'] <= end_day)
        filtered_df_endDate = df[mask_endDate]

        # Filter the dataframe for min_startDate
        mask_startDate = (df['startDate'] >= start_night) & (df['startDate'] <= end_night)
        filtered_df_startDate = df[mask_startDate]

        # Find max_endDate and min_startDate
        max_endDate = filtered_df_endDate['endDate'].max()
        min_startDate = filtered_df_startDate['startDate'].min()

        # Append the results to the list
        results.append({
            'date': date,
            'startSleep': max_endDate,
            'endSleep': min_startDate
        })

    # Convert the results to a dataframe and return
    result_df = pd.DataFrame(results)
    
    result_df["night_hours"] = (result_df["endSleep"] - result_df["startSleep"]).dt.total_seconds() / 3600
    
    return result_df

result_df = find_sleep_intervals(df)
print(result_df)

          date          startSleep            endSleep  hours_between
0   2020-09-25 2020-09-26 02:55:00 2020-09-26 07:55:00       5.000000
1   2020-09-26 2020-09-26 23:35:00 2020-09-27 07:20:00       7.750000
2   2020-09-27 2020-09-28 00:25:00 2020-09-28 06:15:00       5.833333
3   2020-09-28 2020-09-28 23:05:00 2020-09-29 07:35:00       8.500000
4   2020-09-29 2020-09-29 22:35:00 2020-09-30 07:00:00       8.416667
..         ...                 ...                 ...            ...
899 2023-03-13 2023-03-14 02:40:00 2023-03-14 10:00:00       7.333333
900 2023-03-14 2023-03-15 01:35:00 2023-03-15 07:40:00       6.083333
901 2023-03-15 2023-03-16 00:30:00 2023-03-16 04:25:00       3.916667
902 2023-03-16 2023-03-16 23:45:00 2023-03-17 04:40:00       4.916667
903 2023-03-17 2023-03-17 23:45:00                 NaT            NaN

[904 rows x 4 columns]


In [29]:
import pandas as pd

df = pd.read_csv('./data/xml_export/AppleStandTime.csv', low_memory=False)

# Assuming your dataframe is called df and has columns 'startDate' and 'endDate'
# Make sure the columns are in datetime format
df['startDate'] = pd.to_datetime(df['startDate']).dt.tz_localize(None)
df['endDate'] = pd.to_datetime(df['endDate']).dt.tz_localize(None)

df['date'] = (df['startDate'] - pd.Timedelta(hours=12)).dt.date

df["hours_between"] = (df["startDate"] - df["endDate"].shift(1)).dt.total_seconds() / 3600

# filter out hours start or end between > 12
df = df[df["hours_between"] < 12]

# Group by date, max hours between
df = df.groupby('date').agg(hours_between=('hours_between', 'max')).reset_index()
print(df)

           date  hours_between
0    2020-09-25       5.000000
1    2020-09-26       7.750000
2    2020-09-27       5.833333
3    2020-09-28       8.500000
4    2020-09-29       8.750000
..          ...            ...
896  2023-03-13       8.250000
897  2023-03-14       6.083333
898  2023-03-15       3.916667
899  2023-03-16       5.000000
900  2023-03-17       4.083333

[901 rows x 2 columns]


In [30]:
truth = pd.read_csv('./data/train.csv', low_memory=False)
truth["date"] = pd.to_datetime(truth["date"]).dt.date

print(truth)

#merge trith with df on date
df = pd.merge(df, truth, on='date', how='left')

df = df.dropna()



            date  sleep_hours
0     2015-02-19     6.400000
1     2015-02-20     7.583333
2     2015-02-21     6.350000
3     2015-02-22     6.500000
4     2015-02-23     8.916667
...          ...          ...
2349  2021-12-25     7.933333
2350  2021-12-26     3.850000
2351  2021-12-29     6.175000
2352  2021-12-30     5.158333
2353  2021-12-31     5.908333

[2354 rows x 2 columns]


In [31]:
import numpy as np

# RMSE with sleep_hours and hours_between_start  
from sklearn.metrics import mean_squared_error

rmse_start = np.sqrt(mean_squared_error(df["sleep_hours"], df["hours_between"]))

print(rmse_start)


1.8476136230416695
